In [1]:
import re
import time
import copy
import itertools


def a(part):
    #parse input
    i=0
    floors=[[] for i in range(4)]
    for l in open("11.txt").readlines():
        for x in re.findall("([a-z]+-compatible microchip|[a-z]+ generator)",l):
            #print(x)
            x=x.replace("prom","x")
            if "compatible" in x:
                floors[i].append(x[0]+'M')
            else:
                floors[i].append(x[0]+'G')
        i+=1
    if part == 2:
        #used prstx
        floors[0].append('aG')
        floors[0].append('aM')
        floors[0].append('bG')
        floors[0].append('bM')
    
    
    for x,f in list(enumerate(floors))[::-1]:
        print("F%d"%(x+1), f)

    #Make index
    all=list(itertools.chain.from_iterable(floors))
    all = sorted(all, key=lambda x: x[0])
    all.append('E')
    allcnt = len(all)

    e2i = dict([(y,x) for x,y in enumerate(all)])
    i2e = dict([(x,y) for x,y in enumerate(all)])
    print(e2i)
    print(i2e)



    #convert floor to index form
    start = [0 for x in range(len(all))] 
    for f in range(len(floors)):
        for x in floors[f]:
            start[e2i[x]]=f
    print(start)
    
    
    def collapse(s):
        #print("collapse")
        #print(s)
        elements = s[:-1] 
        elevator = s[-1]
        pairs = []
        for i in list(range(len(elements)))[::2]:
            pairs.append([elements[i],elements[i+1]])
        #print(pairs)
        pairs.sort(key=lambda x: x[0]*10+x[1])
        #print(pairs)
        
        return list(itertools.chain.from_iterable(pairs)) + [elevator]
        
    #collapse(start)
    #return -1

    print("the situation space is ", 4**len(start))

    #situations can be stored in a single integer

    def int2state(i, l=allcnt):
        r = []
        while len(r) < l:
            r.insert(0, i % 4)
            i = i // 4
        return r

    
    def state2int(s, l=allcnt):
        r = 0
        for y in range(l):
            r *= 4
            r += s[y]
        return r

    def print_state(s):
        for f in range(4)[::-1]:
            r="F%d "%(f+1)
            for x,i in enumerate(s):
                if i == f:
                    r += i2e[x]
                else:
                    r+= "  "
                r+= " "
            print(r)

    print_state(start)


    def h(s):
        r = 0
        for i in s:
            r += 3-i
        return r

    def f(sit):
        state, cost = sit
        return h(state)+cost

    def isgoal(sit):
        for i in range(len(sit)-1):
            if sit[i] != 3:
                return False
        return True

    def floorcomb(f):
        lst= list(itertools.combinations(f,1))
        lst+=list(itertools.combinations(f,2))
        return lst

    def fry(state):
        #walk over all chips
        for ix, chipfloor in list(enumerate(state[:-1]))[1:-1:2]:
            #if their generator is not on the same floor
            if chipfloor != state[ix-1]:
                #they fry if other generators are on same floor
                if chipfloor in state[0:-1:2]:
                    return True
        return False


    start_situation = (start,0)

    futures={}
    futures[f(start_situation)] = [(state2int(start), 0)]
    past = {}


    counter = 0
    nextprint = 0
    searching = True
    while searching:
        n = time.time()
        if n > nextprint:
            print("n:%d p:%d f:%d"%(counter, len(past.keys()), sum([len(futures[x]) for x in futures.keys()])), end='\r')
            nextprint = n+1
        counter += 1

        #take out next situation
        fix = min(futures.keys())
        state_int,cost = futures[fix][0]
        futures[fix]=futures[fix][1:]
        if len(futures[fix]) == 0:
            del futures[fix]


        if state_int not in past:
            past[state_int]=1

            state = int2state(state_int)

            if isgoal(state):
                print("\n",state, cost, "is goal (a)")
                return cost

            #print(state)
            #walk through all combinations of single and double moves from elevator floor
            elevator = state[-1]
            moveindexes = [x for x,y in list(enumerate(state))[:-1] if y == elevator]
            #print(elevator, moveindexes)
            nextfloors=[]
            if elevator > 0:
                nextfloors.append(elevator-1)
            if elevator < 3:
                nextfloors.append(elevator+1)
            for nf in nextfloors:
                for m in floorcomb(moveindexes):
                    state2 = copy.deepcopy(state)
                    for mi in m:
                        state2[mi]=nf
                    #remember elevator
                    state2[-1]=nf
                    sit2 = (state2, cost+1)

                    if not fry(state2):
                        if isgoal(state2):
                            print("\n",state2, cost+1, "is goal (B)")
                            return cost+1
                        state2= collapse(state2)
                        s2i = state2int(state2)
                        if s2i not in past:
                            fix = f(sit2)
                            if not fix in futures:
                                futures[fix]=[]
                            futures[fix].append((s2i, cost+1))
                    #else:
                    #    print("fry")
                    #    print_state(state2)

print("part1", a(1))
print("part2", a(2))

    


F4 []
F3 ['xG', 'xM', 'rG', 'rM']
F2 ['pM', 'sM']
F1 ['tG', 'tM', 'pG', 'sG']
{'pG': 0, 'pM': 1, 'rG': 2, 'rM': 3, 'sG': 4, 'sM': 5, 'tG': 6, 'tM': 7, 'xG': 8, 'xM': 9, 'E': 10}
{0: 'pG', 1: 'pM', 2: 'rG', 3: 'rM', 4: 'sG', 5: 'sM', 6: 'tG', 7: 'tM', 8: 'xG', 9: 'xM', 10: 'E'}
[0, 1, 2, 2, 0, 1, 0, 0, 2, 2, 0]
the situation space is  4194304
F4                                  
F3       rG rM             xG xM    
F2    pM          sM                
F1 pG          sG    tG tM       E 
n:0 p:0 f:1
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 31 is goal (B)
part1 31
F4 []
F3 ['xG', 'xM', 'rG', 'rM']
F2 ['pM', 'sM']
F1 ['tG', 'tM', 'pG', 'sG', 'aG', 'aM', 'bG', 'bM']
{'aG': 0, 'aM': 1, 'bG': 2, 'bM': 3, 'pG': 4, 'pM': 5, 'rG': 6, 'rM': 7, 'sG': 8, 'sM': 9, 'tG': 10, 'tM': 11, 'xG': 12, 'xM': 13, 'E': 14}
{0: 'aG', 1: 'aM', 2: 'bG', 3: 'bM', 4: 'pG', 5: 'pM', 6: 'rG', 7: 'rM', 8: 'sG', 9: 'sM', 10: 'tG', 11: 'tM', 12: 'xG', 13: 'xM', 14: 'E'}
[0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 0, 0, 2, 2, 0]
the situat